# YOLO-NAS Training

Following the fine tuning example notebook here: https://github.com/Deci-AI/super-gradients/blob/master/YOLONAS.md

In [21]:
%%capture
!pip install super-gradients==3.2.0
!pip install imutils
!pip install roboflow
!pip install pytube --upgrade

In [22]:
#Using smallest model, see https://docs.ultralytics.com/models/yolo-nas/#overview

from super_gradients.training import models

yolo_nas_s = models.get("yolo_nas_s", pretrained_weights="coco")

[2024-05-15 13:59:02] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2024-05-15 13:59:02] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_s


In [23]:
#Printing the model summary

!pip install torchinfo
from torchinfo import summary

summary(model=yolo_nas_s,
        input_size=(16, 3, 640, 640),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Layer (type (var_name))                                           Input Shape          Output Shape         Param #              Trainable
YoloNAS_S (YoloNAS_S)                                             [16, 3, 640, 640]    [16, 8400, 4]        --                   True
├─NStageBackbone (backbone)                                       [16, 3, 640, 640]    [16, 96, 160, 160]   --                   True
│    └─YoloNASStem (stem)                                         [16, 3, 640, 640]    [16, 48, 320, 320]   --                   True
│    │    └─QARepVGGBlock (conv)                                  [16, 3, 640, 640]    [16, 48, 320, 320]   3,024                True
│    └─YoloNASStage (stage1)                                      [16, 48, 320, 320]   [16, 96, 160, 160]   --                   True
│    │    └─QARepVGGBlock (downsample)                            [16, 48, 320, 320]   [16, 96, 160, 160]   88,128               True
│    │    └─YoloNASCSPLayer (blocks)                     

In [24]:
from super_gradients.training import Trainer

CHECKPOINT_DIR = '/projectnb/npbssmic/ac25/defect_training_yolonas'
trainer = Trainer(experiment_name='train1(5_15)', ckpt_root_dir=CHECKPOINT_DIR)

In [25]:
from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val

In [26]:
dataset_params = {
    'data_dir':'/projectnb/npbssmic/ac25/Dataset_Preparation/results',
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'classes': ['Defect']
}

In [27]:
#Creating datasets and dataloaders for training and validation

from IPython.display import clear_output

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':16,
        'num_workers':2
    }
)

clear_output()

Indexing dataset annotations: 100%|██████████| 572/572 [00:00<00:00, 1622.99it/s]


In [28]:
train_data.dataset.transforms

[DetectionMosaic('additional_samples_count': 3, 'non_empty_targets': False, 'prob': 1.0, 'input_dim': (640, 640), 'enable_mosaic': True, 'border_value': 114),
 DetectionRandomAffine('additional_samples_count': 0, 'non_empty_targets': False, 'degrees': 10.0, 'translate': 0.1, 'scale': [0.1, 2], 'shear': 2.0, 'target_size': (640, 640), 'enable': True, 'filter_box_candidates': True, 'wh_thr': 2, 'ar_thr': 20, 'area_thr': 0.1, 'border_value': 114),
 DetectionMixup('additional_samples_count': 1, 'non_empty_targets': True, 'input_dim': (640, 640), 'mixup_scale': [0.5, 1.5], 'prob': 1.0, 'enable_mixup': True, 'flip_prob': 0.5, 'border_value': 114),
 DetectionHSV('additional_samples_count': 0, 'non_empty_targets': False, 'prob': 1.0, 'hgain': 5, 'sgain': 30, 'vgain': 30, 'bgr_channels': (0, 1, 2), '_additional_channels_warned': False),
 DetectionHorizontalFlip('additional_samples_count': 0, 'non_empty_targets': False, 'prob': 0.5),
 DetectionPaddedRescale('additional_samples_count': 0, 'non_em

In [29]:
#keeping only the last two transforms: PaddedRescale, FormatTransform
train_data.dataset.transforms = train_data.dataset.transforms[-2:]
train_data.dataset.transforms


[DetectionPaddedRescale('additional_samples_count': 0, 'non_empty_targets': False, 'swap': (2, 0, 1), 'input_dim': (640, 640), 'pad_value': 114),
 DetectionTargetsFormatTransform('additional_samples_count': 0, 'non_empty_targets': False, 'input_format': OrderedDict([('bboxes', name=bboxes length=4 format=<super_gradients.training.datasets.data_formats.bbox_formats.xyxy.XYXYCoordinateFormat object at 0x1514c5549810>), ('labels', name=labels length=1)]), 'output_format': OrderedDict([('labels', name=labels length=1), ('bboxes', name=bboxes length=4 format=<super_gradients.training.datasets.data_formats.bbox_formats.cxcywh.CXCYWHCoordinateFormat object at 0x1514c5580610>)]), 'min_bbox_edge_size': 1, 'input_dim': (640, 640), 'targets_format_converter': <super_gradients.training.datasets.data_formats.format_converter.ConcatenatedTensorFormatConverter object at 0x1514acc566b0>)]

In [30]:
#Visualizing dataset after transformations applied. Different colors do not correspond to different classes

train_data.dataset.plot()

In [31]:
#Get model for training - using smallest version

from super_gradients.training import models
model = models.get('yolo_nas_s',
                   num_classes=len(dataset_params['classes']),
                   pretrained_weights="coco"
                   )

[2024-05-15 14:00:03] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2024-05-15 14:00:03] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_s


In [32]:
#Setting training parameters and metrics to watch 

from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

train_params = {
    # DISABLING SILENT MODE
    'silent_mode': False,
    "average_best_models":False,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    # ONLY TRAINING FOR 10 EPOCHS FOR THIS EXAMPLE NOTEBOOK
    "max_epochs": 10,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        # NOTE: num_classes needs to be defined here
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            # NOTE: num_classes needs to be defined here
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

In [34]:
#Start training 

trainer.train(model=model,
              training_params=train_params,
              train_loader=train_data,
              valid_loader=val_data)

The console stream is now moved to /projectnb/npbssmic/ac25/defect_training_yolonas/train1(5_15)/console_May15_14_00_39.txt


[2024-05-15 14:00:39] INFO - sg_trainer.py - Using EMA with params {'decay': 0.9, 'decay_type': 'threshold'}
[2024-05-15 14:00:40] INFO - sg_trainer_utils.py - TRAINING PARAMETERS:
    - Mode:                         Single GPU
    - Number of GPUs:               1          (1 available on the machine)
    - Dataset size:                 3724       (len(train_set))
    - Batch size per GPU:           16         (batch_size)
    - Batch Accumulate:             1          (batch_accumulate)
    - Total batch size:             16         (num_gpus * batch_size)
    - Effective Batch size:         16         (num_gpus * batch_size * batch_accumulate)
    - Iterations per epoch:         232        (len(train_loader))
    - Gradient updates per epoch:   232        (len(train_loader) / batch_accumulate)

[2024-05-15 14:00:40] INFO - sg_trainer.py - Started training for 10 epochs (0/9)

Validating: 100%|██████████| 36/36 [00:03<00:00, 10.72it/s]
[2024-05-15 14:01:34] INFO - base_sg_logger.py -

SUMMARY OF EPOCH 0
├── Train
│   ├── Ppyoloeloss/loss_cls = 1.8096
│   ├── Ppyoloeloss/loss_iou = 0.4633
│   ├── Ppyoloeloss/loss_dfl = 1.787
│   └── Ppyoloeloss/loss = 3.8613
└── Validation
    ├── Ppyoloeloss/loss_cls = 1.762
    ├── Ppyoloeloss/loss_iou = 0.4861
    ├── Ppyoloeloss/loss_dfl = 1.7398
    ├── Ppyoloeloss/loss = 3.8471
    ├── Precision@0.50 = 0.0
    ├── Recall@0.50 = 0.0
    ├── Map@0.50 = 0.0026
    └── F1@0.50 = 0.0



Validating epoch 1: 100%|██████████| 36/36 [00:03<00:00, 10.09it/s]
[2024-05-15 14:02:28] INFO - base_sg_logger.py - Checkpoint saved in /projectnb/npbssmic/ac25/defect_training_yolonas/train1(5_15)/ckpt_best.pth
[2024-05-15 14:02:28] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.21102741360664368
Train epoch 2:   0%|          | 0/232 [00:00<?, ?it/s]

SUMMARY OF EPOCH 1
├── Train
│   ├── Ppyoloeloss/loss_cls = 1.1126
│   │   ├── Epoch N-1      = 1.8096 (↘ -0.697)
│   │   └── Best until now = 1.8096 (↘ -0.697)
│   ├── Ppyoloeloss/loss_iou = 0.3154
│   │   ├── Epoch N-1      = 0.4633 (↘ -0.1479)
│   │   └── Best until now = 0.4633 (↘ -0.1479)
│   ├── Ppyoloeloss/loss_dfl = 1.1734
│   │   ├── Epoch N-1      = 1.787  (↘ -0.6136)
│   │   └── Best until now = 1.787  (↘ -0.6136)
│   └── Ppyoloeloss/loss = 2.4878
│       ├── Epoch N-1      = 3.8613 (↘ -1.3736)
│       └── Best until now = 3.8613 (↘ -1.3736)
└── Validation
    ├── Ppyoloeloss/loss_cls = 1.0814
    │   ├── Epoch N-1      = 1.762  (↘ -0.6806)
    │   └── Best until now = 1.762  (↘ -0.6806)
    ├── Ppyoloeloss/loss_iou = 0.3186
    │   ├── Epoch N-1      = 0.4861 (↘ -0.1675)
    │   └── Best until now = 0.4861 (↘ -0.1675)
    ├── Ppyoloeloss/loss_dfl = 1.14
    │   ├── Epoch N-1      = 1.7398 (↘ -0.5998)
    │   └── Best until now = 1.7398 (↘ -0.5998)
    ├── Ppyoloeloss/loss =

...

Validating epoch 8: 100%|██████████| 36/36 [00:03<00:00, 11.28it/s]


SUMMARY OF EPOCH 8
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.6079
│   │   ├── Epoch N-1      = 0.6853 (↘ -0.0774)
│   │   └── Best until now = 0.6853 (↘ -0.0774)
│   ├── Ppyoloeloss/loss_iou = 0.1555
│   │   ├── Epoch N-1      = 0.1831 (↘ -0.0277)
│   │   └── Best until now = 0.1831 (↘ -0.0277)
│   ├── Ppyoloeloss/loss_dfl = 0.7266
│   │   ├── Epoch N-1      = 0.7947 (↘ -0.068)
│   │   └── Best until now = 0.7947 (↘ -0.068)
│   └── Ppyoloeloss/loss = 1.3598
│       ├── Epoch N-1      = 1.5405 (↘ -0.1806)
│       └── Best until now = 1.5405 (↘ -0.1806)
└── Validation
    ├── Ppyoloeloss/loss_cls = 1.1347
    │   ├── Epoch N-1      = 1.0977 (↗ 0.037)
    │   └── Best until now = 1.0557 (↗ 0.079)
    ├── Ppyoloeloss/loss_iou = 0.3423
    │   ├── Epoch N-1      = 0.3299 (↗ 0.0124)
    │   └── Best until now = 0.3015 (↗ 0.0408)
    ├── Ppyoloeloss/loss_dfl = 1.2385
    │   ├── Epoch N-1      = 1.1862 (↗ 0.0523)
    │   └── Best until now = 1.0792 (↗ 0.1593)
    ├── Ppyoloeloss/loss = 2.609

Validating epoch 9: 100%|██████████| 36/36 [00:03<00:00, 11.09it/s]


SUMMARY OF EPOCH 9
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.5577
│   │   ├── Epoch N-1      = 0.6079 (↘ -0.0502)
│   │   └── Best until now = 0.6079 (↘ -0.0502)
│   ├── Ppyoloeloss/loss_iou = 0.1315
│   │   ├── Epoch N-1      = 0.1555 (↘ -0.0239)
│   │   └── Best until now = 0.1555 (↘ -0.0239)
│   ├── Ppyoloeloss/loss_dfl = 0.6733
│   │   ├── Epoch N-1      = 0.7266 (↘ -0.0533)
│   │   └── Best until now = 0.7266 (↘ -0.0533)
│   └── Ppyoloeloss/loss = 1.2232
│       ├── Epoch N-1      = 1.3598 (↘ -0.1366)
│       └── Best until now = 1.3598 (↘ -0.1366)
└── Validation
    ├── Ppyoloeloss/loss_cls = 1.1817
    │   ├── Epoch N-1      = 1.1347 (↗ 0.047)
    │   └── Best until now = 1.0557 (↗ 0.126)
    ├── Ppyoloeloss/loss_iou = 0.3497
    │   ├── Epoch N-1      = 0.3423 (↗ 0.0074)
    │   └── Best until now = 0.3015 (↗ 0.0482)
    ├── Ppyoloeloss/loss_dfl = 1.2667
    │   ├── Epoch N-1      = 1.2385 (↗ 0.0282)
    │   └── Best until now = 1.0792 (↗ 0.1874)
    ├── Ppyoloeloss/loss = 2.6

[2024-05-15 14:09:36] INFO - base_sg_logger.py - [CLEANUP] - Successfully stopped system monitoring process
